In [ ]:
import gcsfs
import xarray as xr
import zarr
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio.features import rasterize
from skimage.measure import block_reduce
import time
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
from satextractor.utils import tqdm_joblib

from oxeo.water.models.utils import merge_masks
import gcsfs
fs = gcsfs.GCSFileSystem(project='oxeo-main')

In [ ]:
import geopandas as gpd
shapefile = gpd.read_file("../../ReservoirWatchHack/Data/Contextual_Shapefiles/reservoirs.shp")



In [ ]:
shapefile.geometry.to_file('india_reservoirs.geojson', driver='GeoJSON')  

In [ ]:
patches_paths = fs.ls('oxeo-water/eo/harangi')
patches_paths



In [ ]:
full_mask,dates= merge_masks(patches_paths, 1000,datetime(2018,2,9),datetime(2019,2,9))
plt.imshow(full_mask[40])
plt.show()

In [ ]:
shapefile.iloc[3].geometry